In [5]:
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score

import numpy as np
import tensorflow as tf

import keras
import os
import sys

sys.path.append('..')
import data
import model
import preprocessing
import utils

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Cuarto paso: correr el modelo

In [8]:
l_rates = [0.001, 0.01]
epochs = [10, 20, 30] 
batch_sizes = [16, 32]

experiment = 1 #Número donde inician los experimentos
for batch in batch_sizes:
    for epoch in epochs:
        for learning_rate in l_rates:
        
            print("*"*20)
            print("EXPERIMENTO: Batch:", batch, "Epoch: ", epoch, "Learning rate: ", learning_rate)
            
            #Cargar los parametros cada que inicia un experimento
            params = utils.yaml_to_dict(os.path.join('..','config.yml'))
            params['data_dir'] = os.path.join('..',params['data_dir'])
            
            experiment_name = 'Experimento1-Configuracion'+str(experiment)
            params['model_dir'] = os.path.join('..','experiments-weights',experiment_name)
            params['batch_size'] = batch
            params['num_epochs'] = epoch
            params['learning_rate'] = learning_rate
            
            #Limpiar Sesión
            tf.keras.backend.clear_session()
            
            #Cargar el modelo
            inputs = tf.keras.layers.Input(shape=(110,480,3))
            net = model.ModelArchitecture(num_classes=params['num_classes'])
            x = net(inputs, training=False)
            
            #Generar conjuntos de entrenamiento y validación y llevarlos a un generador
            preprocessing.split_data(params)
            train_generator = data.make_datagenerator(params, mode='training')
            val_generator = data.make_datagenerator(params,mode='validation')
            
            #Crear el directorio que guarda los checkpoints
            if not os.path.exists(params['model_dir']):
                os.makedirs(params['model_dir'])

            #Definir los callbacks
            cp_callback = tf.keras.callbacks.ModelCheckpoint(
                os.path.join(params['model_dir'], 'tf_ckpt'), 
                save_weights_only=True, 
                verbose=1,
                period=5)

            tb_callback = tf.keras.callbacks.TensorBoard(
                os.path.join(params['model_dir'], 'logs'))

            #Definir la función de optimización
            optimizer = tf.keras.optimizers.Adam(params['learning_rate'])

            #Definir los pasos para entrenamiento y validación
            steps_per_epoch = train_generator.n // params['batch_size']
            validation_steps = val_generator.n // params['batch_size']

            #Entrenar el modelo
            net.compile(optimizer=optimizer, loss=params['loss'], metrics=['sparse_categorical_accuracy'])
            net.fit_generator(
                train_generator, 
                steps_per_epoch=steps_per_epoch, 
                epochs=params['num_epochs'],
                workers=4,
                validation_data=val_generator, 
                validation_steps=validation_steps,
                callbacks=[cp_callback,tb_callback])
            
    
            #Obtener las métricas de resultados en el conjunto de validación
            params['batch_size'] = val_generator.n
            params['shuffle'] = False
            val_generator = data.make_datagenerator(params,mode='validation')
            Y_pred = net.predict_generator(val_generator, steps=1)
            y_pred = np.argmax(Y_pred, axis=1)
            
            print(confusion_matrix(val_generator.data, y_pred))
            target_names = ["arranca", "acelera", "pita", "izquierda","frena", "detente", "retrocede", "gira", "avanza", "derecha"]
            print(classification_report(val_generator.data, y_pred, target_names=target_names))
            
            #Aumentar el contador de experimentos
            experiment += 1

********************
EXPERIMENTO: Batch: 2 Epoch:  10 Learning rate:  0.001
Found 2958 images.


W0729 23:35:37.519514 139898620475200 callbacks.py:875] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.


Found 30 images.
Epoch 1/10


ResourceExhaustedError: OOM when allocating tensor of shape [422400,128] and type float
	 [[node Adam/model_architecture/dense/kernel/v/Initializer/zeros (defined at <ipython-input-8-06db4b6d6670>:66) ]]

Original stack trace for 'Adam/model_architecture/dense/kernel/v/Initializer/zeros':
  File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/manuela/.python_envs/pi2/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/manuela/.python_envs/pi2/lib/python3.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/manuela/.python_envs/pi2/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/home/manuela/.python_envs/pi2/lib/python3.7/site-packages/tornado/platform/asyncio.py", line